In [74]:
import pandas as pd
import json
import copy
import csv
import requests
import os
from dotenv import load_dotenv

load_dotenv()

True

In [75]:
music_items= []
final_list = []
# Get the value of 'HOME'
# environment variable
env_key = 'BEARER_TOKEN'
token = os.getenv(env_key)


In [76]:
def generate():
    with open('pop-playlist.json', 'r') as json_file:
        json_data = json.load(json_file)
        music_items = json_data["items"].copy()
        # print(music_items[0])

        for item in music_items:
            final_list.append([
                item["track"]["id"], 
                item["track"]["artists"][0]["name"],
                item["track"]["name"]
                
                ])

In [77]:
def create():
    with open("custom_playlist-pop.csv", "w", newline="", encoding="UTF8") as csv_file:
            # create the csv writer
            writer = csv.writer(csv_file)
            writer.writerow(["track_id", "artist", "music"])
            # write a row to the csv file
            for music in final_list :
                writer.writerow(music)

In [78]:
def get_track_ids():
    music_ids = ''
    for id in final_list:
        music_ids += ''+id[0]+','

    return music_ids

In [79]:
def get_audio_tracks_specs():
    auth_token=token
    hed = {'Authorization': 'Bearer ' + auth_token}
    data = {"ids" : get_track_ids()}

    url = 'https://api.spotify.com/v1/audio-features'
    response = requests.get(url, params=data, headers=hed)
    # print(response.json())
    return response.json()

In [80]:
def update_dataset():

    specs_dict = {"danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "duration_ms"}

    music_specs = get_audio_tracks_specs()['audio_features']
    with open("music_playlist-pop.csv", "w", newline='', encoding="UTF8") as csv_file:
            # create the csv writer
            writer = csv.writer(csv_file)
            writer.writerow(["track_id", "artist", "music", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "duration_ms"])
            # write a row to the csv file
            for index, music in enumerate(final_list) :
                new_music = music.copy()
                new_specs = [music_specs[index][val] for val in music_specs[index].keys() if val in specs_dict]
                new_music += new_specs
                writer.writerow(new_music)

In [81]:
generate()
create()
update_dataset()

In [82]:
dataset =pd.read_csv("music_playlist-pop.csv")


dataset.info();


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          100 non-null    object 
 1   artist            100 non-null    object 
 2   music             100 non-null    object 
 3   danceability      100 non-null    float64
 4   energy            100 non-null    float64
 5   loudness          100 non-null    float64
 6   speechiness       100 non-null    float64
 7   acousticness      100 non-null    float64
 8   instrumentalness  100 non-null    float64
 9   liveness          100 non-null    float64
 10  duration_ms       100 non-null    int64  
dtypes: float64(7), int64(1), object(3)
memory usage: 8.7+ KB
